In [17]:
import os

! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage

In [18]:
PROJECT_ID = "aa-ai-specialisation"
REGION = "us-central1"
PRIVATE_REPO = "bf-repo"
!gcloud config set project {PROJECT_ID}

Updated property [core/project].


In [19]:
BUCKET_URI = "gs://specialization_black_friday"

In [20]:
from google.cloud import aiplatform

In [33]:
%%writefile Dockerfile
FROM python:3.11-slim
WORKDIR /root

RUN apt-get update
RUN apt-get -y install curl
RUN curl https://dl.google.com/dl/cloudsdk/release/google-cloud-sdk.tar.gz > /tmp/google-cloud-sdk.tar.gz

# Installing the package
RUN mkdir -p /usr/local/gcloud \
  && tar -C /usr/local/gcloud -xvf /tmp/google-cloud-sdk.tar.gz \
  && /usr/local/gcloud/google-cloud-sdk/install.sh

# Adding the package path to local
ENV PATH $PATH:/usr/local/gcloud/google-cloud-sdk/bin

RUN python -m pip install 'xgboost==1.6.2' 'scikit-learn' 'pandas' 'numpy'

RUN echo '[GoogleCompute]\nservice_account = default' > /etc/boto.cfg

# Copies the trainer code
RUN mkdir /root/trainer
COPY trainer/train.py /root/trainer/train.py

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python3.11", "trainer/train.py"]

Overwriting Dockerfile


In [11]:
!gcloud services enable artifactregistry.googleapis.com

In [23]:
TRAIN_IMAGE = (
    f"{REGION}-docker.pkg.dev/" + PROJECT_ID + f"/{PRIVATE_REPO}" + "/bfd"
)
print("Deployment:", TRAIN_IMAGE)

Deployment: us-central1-docker.pkg.dev/aa-ai-specialisation/bf-repo/bfd


In [34]:
!docker build -t 'us-central1-docker.pkg.dev/aa-ai-specialisation/bf-repo/bfd' .

Sending build context to Docker daemon   1.04GB
Step 1/12 : FROM python:3.11-slim
 ---> dd150e5400f1
Step 2/12 : WORKDIR /root
 ---> Using cache
 ---> 58397438673e
Step 3/12 : RUN apt-get update
 ---> Using cache
 ---> 7da4d78a035f
Step 4/12 : RUN apt-get -y install curl
 ---> Using cache
 ---> 4d3107282599
Step 5/12 : RUN curl https://dl.google.com/dl/cloudsdk/release/google-cloud-sdk.tar.gz > /tmp/google-cloud-sdk.tar.gz
 ---> Using cache
 ---> 3af59f3152ee
Step 6/12 : RUN mkdir -p /usr/local/gcloud   && tar -C /usr/local/gcloud -xvf /tmp/google-cloud-sdk.tar.gz   && /usr/local/gcloud/google-cloud-sdk/install.sh
 ---> Using cache
 ---> 6db30296d46a
Step 7/12 : ENV PATH $PATH:/usr/local/gcloud/google-cloud-sdk/bin
 ---> Using cache
 ---> 1160eb59132a
Step 8/12 : RUN python -m pip install 'xgboost==1.6.2' 'scikit-learn' 'pandas' 'numpy'
 ---> Running in 90275f6c39a8
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.9/255.9 MB 4.2 MB/s eta 0:00:00
  Obtaining dependency information for s

In [35]:
!docker push 'us-central1-docker.pkg.dev/aa-ai-specialisation/bf-repo/bfd'

Using default tag: latest
The push refers to repository [us-central1-docker.pkg.dev/aa-ai-specialisation/bf-repo/bfd]

82204c05: Preparing 
1d47cfd1: Preparing 
6a68de9d: Preparing 
68119812: Preparing 
67a6907f: Preparing 
4cce5456: Preparing 
63b26753: Preparing 
03e7905c: Preparing 
d8defe69: Preparing 
21973527: Preparing 
4febd598: Preparing 
58ccd7ef: Preparing 
8119812: Pushed   1.032GB/1.024GBPushing  28.36MB/1.024GBlatest: digest: sha256:d5e1c77ac5b3f6aa6a308c5f0e380119e8738a8fd3d258a1102500602d6e84a9 size: 3053


In [28]:
!pip install --upgrade google-auth

In [36]:
!gcloud ai custom-jobs create --region=us-central1 --config=config.yaml --display-name={display_name}

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
CustomJob [projects/653183562498/locations/us-central1/customJobs/7311849837647888384] is submitted successfully.

Your job is still active. You may view the status of your job with the command

  $ gcloud ai custom-jobs describe projects/653183562498/locations/us-central1/customJobs/7311849837647888384

or continue streaming the logs with the command

  $ gcloud ai custom-jobs stream-logs projects/653183562498/locations/us-central1/customJobs/7311849837647888384


In [32]:
!gcloud ai custom-jobs describe projects/653183562498/locations/us-central1/customJobs/7889788333579042816

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
createTime: '2023-12-24T12:54:07.016001Z'
displayName: bfd_xgb_training
endTime: '2023-12-24T12:55:10Z'
error:
  code: 3
  message: 'The replica workerpool0-0 exited with a non-zero status of 1. To find
    out more about why your job exited please check the logs: https://console.cloud.google.com/logs/viewer?project=653183562498&resource=ml_job%2Fjob_id%2F7889788333579042816&advancedFilter=resource.type%3D%22ml_job%22%0Aresource.labels.job_id%3D%227889788333579042816%22'
jobSpec:
  baseOutputDirectory:
    outputUriPrefix: gs://specialization_black_friday
  enableDashboardAccess: true
  enableWebAccess: true
  workerPoolSpecs:
  - containerSpec:
      imageUri: us-central1-docker.pkg.dev/aa-ai-specialisation/bf-repo/bfd
    diskSpec:
      bootDiskSizeGb: 100
      bootDiskType: pd-ssd
    machineSpec:
      machineType: n1-standard-8
    replicaCount: '1'
name: projects/653183562498/locations/us-central1/customJobs/788978